In [ ]:
# ----- MAKE A COPY OF THIS -----

# Description: Uses 07/29/2022 Classification Parameters on other rasters
# Shapefile found in Unsupervised directory
# in classify function, uses sig file from /Unsupervised/Signature/sigfile20220729.gsg
# the "method" variable specifies what type of classification we are using

In [4]:
def get_bands(project, net, date, year): 
    try:
        #fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/R60m/*B[01][1234]_60m.jp2"
        #fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/R60m/*B[01][1234]_60m.jp2"
        fulldir = net + f"/Data/{year}/{date}/GRANULE/L*/IMG_DATA/R60m/*B0[25]_60m.jp2"
        contents = glob.glob(fulldir)
        if not contents:
            #fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/*B[01][1234].jp2"
            #fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/*B[01][284].jp2"
            fulldir = net + f"/Data/{year}/{date}/GRANULE/L*/IMG_DATA/*B0[25].jp2"
            
            contents = glob.glob(fulldir)
    except Exception as e:
        print("An error has occurred:", e)



    match = contents[::-1]

    results = [e[len(e)-12:] for e in match]

    print(results)

    output = f"/{project}/Bands/CompositeBand{date}.tif"
    output = net + output

    
    arcpy.management.CompositeBands(match, output)
    return output

In [5]:
def clip_single_raster(project, net, img, date): 
    # Shapefile is always in the Unsupervised directory (for right now)
    clip_shape = net + "/Unsupervised/ExpandedShape/ExpandedShape.shp"
    clipped_destination = net + f"/{project}/SingleClipped/SingleRaster{date}.tif"
    arcpy.management.Clip(img, clip_shape, clipped_destination)
    return clipped_destination

In [6]:
def train(project, net, img, date):  
    classes = 8
    class_size = 5
    samp_interval = 4
    sig_file = net + f"/{project}/Signature/sigfile{date}.gsg"
    out_unsuper = IsoClusterUnsupervisedClassification(img, classes, class_size, samp_interval, sig_file)
    out_unsuper.save(net + f"/{project}/UnsupervisedTifs/UnsupervisedClassification{date}.tif")
    print("Done training")
    return out_unsuper

In [7]:
def classify(project, net, date):
    raster = net + f"/{project}/SingleClipped/SingleRaster{date}.tif"
    
    # location of current signature file that we are using
    sig_file = net + f"/Unsupervised/Signature/sigfile20220729.gsg"
    
    out_classify = MLClassify(raster, sig_file)
    out_classify.save(net + f"/{project}/Classified/SingleClassify{date}.tif")

In [8]:
def get_table(project, net, date):
    outputTable = net + f"/{project}/Tables/Table{date}.csv"
    inputRaster = net + f"/{project}/Classified/SingleClassify{date}.tif"
    arcpy.ia.SummarizeCategoricalRaster(inputRaster, outputTable)

In [9]:
def get_area(project, net, dates):
    result = {}
    for date in dates:
        path = net + f"/{project}/Tables/Table{date}.csv"
        df = pd.read_csv(path)
        pixelSum = df.sum(axis = 1).values[0]
        pixelWater = df["C_2"].values[0] 
        percentageCovered = (pixelWater/pixelSum) * 100
        result[date] = round(percentageCovered, 4)
    return result

In [50]:
def create_table(net):
    df = pd.DataFrame(columns = ["Year", "X", "Y", "Type", "Bands"])
    directory = net + "/CSV/main.csv"
    if os.path.exists(directory):
        return
    else:
        print("Creating a new table...")
        df.to_csv(directory, index = False)

In [56]:
def save_table(net, obj, year, method, bands):
    create_table(net)
    new_obj = {"X": obj.keys(),
               "Y": obj.values(),
               "Year": year,
               "Type": method,
                "Bands": bands}
    df = pd.DataFrame(new_obj)
    temp = df.to_csv(net + "/CSV/temp.csv", index = False)
    
    print("Appending new information...")
    existing_df = pd.read_csv(net + "/CSV/main.csv")
    new_df = pd.read_csv(net + "/CSV/temp.csv")
  
    existing_df = pd.concat([existing_df, new_df], ignore_index = True)
    existing_df = existing_df.drop_duplicates()

    existing_df.to_csv(net + "/CSV/main.csv", index = False)



In [52]:
# Unsupervised training


import pandas as pd
import matplotlib.pyplot as plt
import arcpy
import os
import glob
from arcpy import env
from arcpy.sa import *
from arcpy.ia import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

arcpy.env.overwriteOutput = True
network_path = "network_path" 
arcpy.CheckOutExtension("ImageAnalyst")

try:
    contents = os.listdir(network_path)
except FileNotFoundError:
    print("NAN")
except Exception as e:
    print("An error has occurred:", e)

#------ CHANGE THESE VARIABLES -----
proj = "2021"
year = "2021"
dates = glob.glob(network_path + f"/Data/{year}/*")
dates = [d[len(d)-8:] for d in dates]
method = "7/29/22SigFile"
bands = "5,2"

In [14]:
## ----- Apply Unsupervised Training on a Raster to get sigFile -----
# training_date = "20220729"
# in_raster = network_path + f"/{proj}/SingleClipped/SingleRaster{training_date}.tif"
# result = train(proj, network_path, in_raster, training_date)
# print("Finished training")


In [15]:
# ----- Getting and Clipping Bands -----
for date in dates:
    out = get_bands(proj, network_path, date, year)
    out_clipped = clip_single_raster(proj, network_path, out, date)
print("Clipped all Rasters")

['0911_B05.jp2', '0911_B02.jp2']
['1809_B05.jp2', '1809_B02.jp2']
['0911_B05.jp2', '0911_B02.jp2']
['0911_B05.jp2', '0911_B02.jp2']
['_B05_60m.jp2', '_B02_60m.jp2']
['1911_B05.jp2', '1911_B02.jp2']
['1809_B05.jp2', '1809_B02.jp2']
['1809_B05.jp2', '1809_B02.jp2']
['0911_B05.jp2', '0911_B02.jp2']
Clipped all Rasters


In [16]:
# ----- Use sig file on all other rasters, and get tables -----
for date in dates:
    res = classify(proj, network_path, date)
    get_table(proj, network_path, date)
    print("Finished", date)
print("Tables saved")

Finished 20210602
Finished 20210610
Finished 20210622
Finished 20210702
Finished 20210712
Finished 20210725
Finished 20210730
Finished 20210809
Finished 20210831
Tables saved


In [57]:
# ----- Do calculations to get the area, and save to csv file -----
obj = get_area(proj, network_path, dates)
method = "7/29/22SigFile"
save_table(network_path, obj, year, method, bands)
print("Added to main table")


Appending new information...
Added to main table


In [ ]:
#----------------------------------------------------- END ------------------------------------------------------------------

In [32]:
# ----- CODE TO PLAY AROUND WITH REGEX -----


import glob
def test(net, date):
    try:
        fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/R60m/*B025_60m.jp2"
        contents = glob.glob(fulldir)
        if not contents:
            fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/*B[01][12].jp2"
            contents = glob.glob(fulldir)
    except Exception as e:
        print("An error has occurred:", e)


    # *B0[234].jp2"   if we want bands 2, 3 4 
    # For directories with 10 20 60

    match = contents[1:len(contents)-1]
    # Need to have bands in descending order for Natural Color : 4 3 2

    results = [e[len(e)-12:] for e in match]
    # Need to have bands in descending order for Natural Color : 4 3 2
    
    print(results)

    return


network_path = "network_path" 
dates = ["20220615", "20220625", "20220710", "20220729", "20220813", "20220828"]
for date in dates:
    test(network_path, date)
    
#2, 3, 4, 11
# Tried 11, 4, 3
# 4, 3, 2
# 8, 3
# 12, 8, 4
# 1, 2, 3
#11, 4, 3, 2

['1809_B02.jp2', '1809_B11.jp2']
['1809_B02.jp2', '1809_B11.jp2']
[]
[]
['5811_B02.jp2', '5811_B11.jp2']
['5759_B02.jp2', '5759_B11.jp2']
